#  <span style="text-align=center; color:DarkMagenta; font-family:Georgia; font-size:1.2em;"> **_M2 - TP EIAH - MAY_** </span>

Eliot Lepoittevin & Benjamin Riviere

# <span style="color:brown; font-family:times; font-size:1.3em;">Code</span>

In [1]:
import pandas as pd
import numpy as np

#load de data from the csv files
df_tr = pd.read_csv ('csv_dump/transition.csv')
df_uf = pd.read_csv ('csv_dump/userfiles.csv')


def LoadPerUserData(dataframe,userNamePos):
    perUser = []
    data = dataframe.values
    
    for i in range(len(data)):
        username = data[i][userNamePos]
        found = False
        for userData in perUser:
            if(userData[0] == username):
                userData[1].append(data[i])
                found = True
        if(found==False):
            perUser.append((username,[data[i]]))
    return perUser


#group the data per user
perUser_tr = LoadPerUserData(df_tr,1)
perUser_uf = LoadPerUserData(df_uf,1)


print(str(perUser_tr))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
def motivation():
    return

In [3]:
def reactivité():
    return

# <span style="color:brown; font-family:times; font-size:1.3em;">Résultats</span>

### <span style="color:green; font-family:Georgia; font-size:1.5em;">END</span>

In [4]:
#Eliot Lepoittevin
#Benjamin Riviere